In [4]:
database = "sqlite_db_pythonsqlite.db"

In [5]:
import sqlite3

In [6]:
conn = sqlite3.connect(database)
print(sqlite3.version)
conn

2.6.0


In [7]:
import pandas as pd

Q1: Some of the facilities charge a fee to members, but some do not.
Write a SQL query to produce a list of the names of the facilities that do.

In [20]:
cur = conn.cursor()
    
query1 = """
    SELECT 
	name,
	membercost
FROM 
	Facilities
WHERE 
	membercost > 0;
    """
cur.execute(query1)

In [21]:
rows = cur.fetchall()

In [22]:
rows

[('Tennis Court 1', 5),
 ('Tennis Court 2', 5),
 ('Massage Room 1', 9.9),
 ('Massage Room 2', 9.9),
 ('Squash Court', 3.5)]

In [27]:
# need to specify column names manually in sqlite3
f = pd.DataFrame(rows, columns=['FacilityName', 'MemberCost'])


In [28]:
f

,FacilityName,MemberCost
0,Tennis Court 1,5.0
1,Tennis Court 2,5.0
2,Massage Room 1,9.9
3,Massage Room 2,9.9
4,Squash Court,3.5


Q2: How many facilities do not charge a fee to members?

In [29]:
cur2 = conn.cursor()
    
query2 = """
    SELECT
	COUNT(membercost)
FROM
	Facilities
WHERE
	membercost = 0;
    """
cur2.execute(query2)

In [30]:
rows2 = cur2.fetchall()

In [35]:
rows2

[(4,)]

Q3: Write an SQL query to show a list of facilities that charge a fee to members,
where the fee is less than 20% of the facility's monthly maintenance cost.
Return the facid, facility name, member cost, and monthly maintenance of the
facilities in question.

In [36]:
cur3 = conn.cursor()
    
query3 = """
SELECT
	membercost,
	monthlymaintenance,
	facid,
	name
FROM
	Facilities
WHERE
	 membercost < (monthlymaintenance  * .2);
    """
cur3.execute(query3)

In [39]:
rows3 = cur3.fetchall()

In [40]:
rows3

[(5, 200, 0, 'Tennis Court 1'),
 (5, 200, 1, 'Tennis Court 2'),
 (0, 50, 2, 'Badminton Court'),
 (0, 10, 3, 'Table Tennis'),
 (9.9, 3000, 4, 'Massage Room 1'),
 (9.9, 3000, 5, 'Massage Room 2'),
 (3.5, 80, 6, 'Squash Court'),
 (0, 15, 7, 'Snooker Table'),
 (0, 15, 8, 'Pool Table')]

In [43]:
f3 = pd.DataFrame(rows3, columns=['Member Cost', 'monthlymaintenance','facid','FacilityName'])
f3

,Member Cost,monthlymaintenance,facid,FacilityName
0,5.0,200,0,Tennis Court 1
1,5.0,200,1,Tennis Court 2
2,0.0,50,2,Badminton Court
3,0.0,10,3,Table Tennis
4,9.9,3000,4,Massage Room 1
5,9.9,3000,5,Massage Room 2
6,3.5,80,6,Squash Court
7,0.0,15,7,Snooker Table
8,0.0,15,8,Pool Table


Q4: Write an SQL query to retrieve the details of facilities with ID 1 and 5.
Try writing the query without using the OR operator.

In [44]:
cur4 = conn.cursor()
    
query4 = """
SELECT 
	*
FROM
	Facilities
WHERE 
	facid IN('1','5')
    """
cur4.execute(query4)

In [45]:
rows4 = cur4.fetchall()

In [46]:
rows4

[(1, 'Tennis Court 2', 5, 25, 8000, 200),
 (5, 'Massage Room 2', 9.9, 80, 4000, 3000)]

In [49]:
f4 = pd.DataFrame(rows4, columns=['monthlymaintenance',
'facid',
'name',
'membercost',
'guestcost',
'initialoutlay'])
f4

,monthlymaintenance,facid,name,membercost,guestcost,initialoutlay
0,1,Tennis Court 2,5.0,25,8000,200
1,5,Massage Room 2,9.9,80,4000,3000


Q5: Produce a list of facilities, with each labelled as 'cheap' or 'expensive', depending on if their monthly maintenance cost is more than $100. Return the name and monthly maintenance of the facilities in question.

In [50]:
cur5 = conn.cursor()
    
query5 = """
SELECT 
	*,
	CASE
		WHEN monthlymaintenance > 100
			THEN 'expensive'
		WHEN monthlymaintenance < 100
			THEN 'cheap' END
		AS type
FROM
	Facilities
ORDER BY
	type
    """
cur5.execute(query5)

In [51]:
rows5 = cur5.fetchall()
rows5

[(2, 'Badminton Court', 0, 15.5, 4000, 50, 'cheap'),
 (3, 'Table Tennis', 0, 5, 320, 10, 'cheap'),
 (6, 'Squash Court', 3.5, 17.5, 5000, 80, 'cheap'),
 (7, 'Snooker Table', 0, 5, 450, 15, 'cheap'),
 (8, 'Pool Table', 0, 5, 400, 15, 'cheap'),
 (0, 'Tennis Court 1', 5, 25, 10000, 200, 'expensive'),
 (1, 'Tennis Court 2', 5, 25, 8000, 200, 'expensive'),
 (4, 'Massage Room 1', 9.9, 80, 4000, 3000, 'expensive'),
 (5, 'Massage Room 2', 9.9, 80, 4000, 3000, 'expensive')]

In [53]:
f5 = pd.DataFrame(rows5, columns=['facid',
'name',
'membercost',
'guestcost',
'initialoutlay',
'monthlymaintenance',
'type'])
f5

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance,type
0,2,Badminton Court,0.0,15.5,4000,50,cheap
1,3,Table Tennis,0.0,5.0,320,10,cheap
2,6,Squash Court,3.5,17.5,5000,80,cheap
3,7,Snooker Table,0.0,5.0,450,15,cheap
4,8,Pool Table,0.0,5.0,400,15,cheap
5,0,Tennis Court 1,5.0,25.0,10000,200,expensive
6,1,Tennis Court 2,5.0,25.0,8000,200,expensive
7,4,Massage Room 1,9.9,80.0,4000,3000,expensive
8,5,Massage Room 2,9.9,80.0,4000,3000,expensive


Q6: You'd like to get the first and last name of the last member(s) who signed up. Try not to use the LIMIT clause for your solution.

In [65]:
cur6 = conn.cursor()
    
query6 = """
SELECT 
	firstname, 
surname, joindate
FROM 
	Members
WHERE 
	joindate = (
		SELECT MAX(joindate)
		FROM Members)
    """
cur6.execute(query6)

In [63]:
rows6 = cur6.fetchall()
rows6

[('Darren', 'Smith', '2012-09-26 18:08:45')]

In [64]:
f6 = pd.DataFrame(rows6, columns=['first name',
                                 'last name',
                                 'joindate'])
f6

,first name,last name,joindate
0,Darren,Smith,2012-09-26 18:08:45


Q7: Produce a list of all members who have used a tennis court. Include in your output the name of the court, and the name of the member formatted as a single column. Ensure no duplicate data, and order by
the member name.

In [96]:
cur7 = conn.cursor()
    
query7 = """
SELECT
  F.name
  ,M.firstname,M.surname
  ,SUM(F.membercost) AS membercost_total
FROM 
  Members M
  INNER JOIN Bookings B ON M.memid = B.memid
  INNER JOIN Facilities F ON B.facid = F.facid
WHERE 
  F.name like '%Tennis Court%'
GROUP BY
  F.name
  ,M.firstname,M.surname
ORDER BY
  M.firstname,M.surname
    """
cur7.execute(query7)



#CASE?

In [97]:
rows7 = cur7.fetchall()
rows7

[('Tennis Court 1', 'Anne', 'Baker', 30),
 ('Tennis Court 2', 'Anne', 'Baker', 175),
 ('Tennis Court 1', 'Burton', 'Tracy', 155),
 ('Tennis Court 2', 'Burton', 'Tracy', 15),
 ('Tennis Court 1', 'Charles', 'Owen', 85),
 ('Tennis Court 2', 'Charles', 'Owen', 205),
 ('Tennis Court 2', 'Darren', 'Smith', 95),
 ('Tennis Court 1', 'David', 'Farrell', 30),
 ('Tennis Court 2', 'David', 'Farrell', 5),
 ('Tennis Court 1', 'David', 'Jones', 125),
 ('Tennis Court 2', 'David', 'Jones', 150),
 ('Tennis Court 1', 'David', 'Pinker', 80),
 ('Tennis Court 1', 'Douglas', 'Jones', 45),
 ('Tennis Court 1', 'Erica', 'Crumpet', 5),
 ('Tennis Court 1', 'Florence', 'Bader', 5),
 ('Tennis Court 2', 'Florence', 'Bader', 40),
 ('Tennis Court 1', 'GUEST', 'GUEST', 500),
 ('Tennis Court 2', 'GUEST', 'GUEST', 565),
 ('Tennis Court 1', 'Gerald', 'Butters', 285),
 ('Tennis Court 2', 'Gerald', 'Butters', 15),
 ('Tennis Court 2', 'Henrietta', 'Rumney', 5),
 ('Tennis Court 1', 'Jack', 'Smith', 110),
 ('Tennis Court 2', '

In [101]:
f7 = pd.DataFrame(rows7, columns=['facility',
'firstname',
'surname',
'membercost_total'])
f7

,facility,firstname,surname,membercost_total
0,Tennis Court 1,Anne,Baker,30
1,Tennis Court 2,Anne,Baker,175
2,Tennis Court 1,Burton,Tracy,155
3,Tennis Court 2,Burton,Tracy,15
4,Tennis Court 1,Charles,Owen,85
5,Tennis Court 2,Charles,Owen,205
6,Tennis Court 2,Darren,Smith,95
7,Tennis Court 1,David,Farrell,30
8,Tennis Court 2,David,Farrell,5
9,Tennis Court 1,David,Jones,125


Q8: Produce a list of bookings on the day of 2012-09-14 which will cost the member (or guest) more than $30. Remember that guests have different costs to members (the listed costs are per half-hour 'slot'), and the guest user's ID is always 0. Include in your output the name of the facility, the name of the member formatted as a single column, and the cost. Order by descending cost, and do not use any subqueries.

In [80]:
cur8 = conn.cursor()
    
query8 = """
SELECT
	starttime,
	monthlymaintenance,
	guestcost,
	b.facid,
    f.facid
FROM 
	Bookings AS b
		INNER JOIN
			Facilities AS f
				ON b.facid = f.facid
WHERE
	starttime LIKE '2012-09-14%'AND 
	(monthlymaintenance > 30 OR 
	guestcost > 30);
    """
cur8.execute(query8)

In [81]:
rows8 = cur8.fetchall()
rows8

[('2012-09-14 08:00:00', 200, 25, 0, 0),
 ('2012-09-14 10:00:00', 200, 25, 0, 0),
 ('2012-09-14 12:30:00', 200, 25, 0, 0),
 ('2012-09-14 14:00:00', 200, 25, 0, 0),
 ('2012-09-14 16:00:00', 200, 25, 0, 0),
 ('2012-09-14 17:30:00', 200, 25, 0, 0),
 ('2012-09-14 19:00:00', 200, 25, 0, 0),
 ('2012-09-14 08:00:00', 200, 25, 1, 1),
 ('2012-09-14 11:00:00', 200, 25, 1, 1),
 ('2012-09-14 14:00:00', 200, 25, 1, 1),
 ('2012-09-14 17:00:00', 200, 25, 1, 1),
 ('2012-09-14 08:00:00', 50, 15.5, 2, 2),
 ('2012-09-14 11:00:00', 50, 15.5, 2, 2),
 ('2012-09-14 13:00:00', 50, 15.5, 2, 2),
 ('2012-09-14 16:00:00', 50, 15.5, 2, 2),
 ('2012-09-14 18:00:00', 50, 15.5, 2, 2),
 ('2012-09-14 08:00:00', 3000, 80, 4, 4),
 ('2012-09-14 09:00:00', 3000, 80, 4, 4),
 ('2012-09-14 11:00:00', 3000, 80, 4, 4),
 ('2012-09-14 12:00:00', 3000, 80, 4, 4),
 ('2012-09-14 13:00:00', 3000, 80, 4, 4),
 ('2012-09-14 14:00:00', 3000, 80, 4, 4),
 ('2012-09-14 16:00:00', 3000, 80, 4, 4),
 ('2012-09-14 18:00:00', 3000, 80, 4, 4),
 ('

In [84]:
f8 = pd.DataFrame(rows8, columns=['starttime',
'monthlymaintenance',
'guestcost',
'facid',
'facid'])
f8.head()

,starttime,monthlymaintenance,guestcost,facid,facid
0,2012-09-14 08:00:00,200,25.0,0,0
1,2012-09-14 10:00:00,200,25.0,0,0
2,2012-09-14 12:30:00,200,25.0,0,0
3,2012-09-14 14:00:00,200,25.0,0,0
4,2012-09-14 16:00:00,200,25.0,0,0


Q9: This time, produce the same result as in Q8, but using a subquery.

In [90]:
cur9 = conn.cursor()
    
query9 = """
SELECT*
FROM(SELECT
	starttime,
	monthlymaintenance,
	guestcost,
	b.facid,
    f.facid
FROM 
	Bookings AS b
		INNER JOIN
			Facilities AS f
				ON b.facid = f.facid
WHERE
	starttime LIKE '2012-09-14%'AND 
	(monthlymaintenance > 30 OR 
	guestcost > 30))
    """
cur9.execute(query9)

In [95]:
rows9 = cur9.fetchall()

In [93]:
f9 = pd.DataFrame(rows9, columns=['starttime',
'monthlymaintenance',
'guestcost',
'facid',
'facid'])
f9.head()

,starttime,monthlymaintenance,guestcost,facid,facid
0,2012-09-14 08:00:00,200,25.0,0,0
1,2012-09-14 10:00:00,200,25.0,0,0
2,2012-09-14 12:30:00,200,25.0,0,0
3,2012-09-14 14:00:00,200,25.0,0,0
4,2012-09-14 16:00:00,200,25.0,0,0


Q10: Produce a list of facilities with a total revenue less than 1000. The output of facility name and total revenue, sorted by revenue. Remember that there's a different cost for guests and members!

In [102]:
cur10 = conn.cursor()
    
query10 = """

SELECT
	name,
	(monthlymaintenance + membercost) AS revenue
FROM 
  Facilities 
WHERE
	monthlymaintenance + membercost < 1000
ORDER BY 
	monthlymaintenance + membercost DESC
    """
cur10.execute(query10)

In [104]:
rows10 = cur10.fetchall()
rows10

[('Tennis Court 1', 205),
 ('Tennis Court 2', 205),
 ('Squash Court', 83.5),
 ('Badminton Court', 50),
 ('Snooker Table', 15),
 ('Pool Table', 15),
 ('Table Tennis', 10)]

In [106]:
f10 = pd.DataFrame(rows10, columns=['facility', 'revenue'])
f10

,facility,revenue
0,Tennis Court 1,205.0
1,Tennis Court 2,205.0
2,Squash Court,83.5
3,Badminton Court,50.0
4,Snooker Table,15.0
5,Pool Table,15.0
6,Table Tennis,10.0


Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order

In [116]:
cur11 = conn.cursor()
    
query11 = """
SELECT 
    m.surname, 
    m.firstname, 
    m.recommendedby AS recomender_id, 
    r.surname AS recomender_lastname, 
    r.firstname AS recomender_firstname
FROM Members AS m
LEFT JOIN Members AS r ON m.recommendedby = r.memid
WHERE m.recommendedby != 0
ORDER BY r.surname, r.firstname;
    """
cur11.execute(query11)

In [117]:
rows11 = cur11.fetchall()

In [115]:
f11 = pd.DataFrame(rows11, columns=['lastname',
'firstname',
'recomender_id',
'recomender_surname',
'recomender_firstname'])
f11

,lastname,firstname,recomender_id,recomender_surname,recomender_firstname
0,GUEST,GUEST,,None,None
1,Smith,Darren,,None,None
2,Smith,Tracy,,None,None
3,Rownam,Tim,,None,None
4,Tracy,Burton,,None,None
5,Farrell,Jemima,,None,None
6,Farrell,David,,None,None
7,Tupperware,Hyacinth,,None,None
8,Smith,Darren,,None,None
9,Sarwin,Ramnaresh,15,Bader,Florence


Q12: Find the facilities with their usage by member, but not guests

In [123]:
cur12 = conn.cursor()
    
query12 = """
SELECT 
    b.facid, 
    COUNT( b.memid ) 
    AS mem_usage, 
    f.name
FROM (SELECT 
        facid, 
        memid
FROM Bookings
WHERE memid !=0) AS b
LEFT JOIN 
    Facilities AS f 
ON b.facid = f.facid
GROUP BY b.facid;
    """
cur12.execute(query12)

In [124]:
rows12 = cur12.fetchall()
rows12

[(0, 308, 'Tennis Court 1'),
 (1, 276, 'Tennis Court 2'),
 (2, 344, 'Badminton Court'),
 (3, 385, 'Table Tennis'),
 (4, 421, 'Massage Room 1'),
 (5, 27, 'Massage Room 2'),
 (6, 195, 'Squash Court'),
 (7, 421, 'Snooker Table'),
 (8, 783, 'Pool Table')]

In [129]:
f12 = pd.DataFrame(rows12, columns=['facid',
'mem_usage',
'name'])
f12

,facid,mem_usage,name
0,0,308,Tennis Court 1
1,1,276,Tennis Court 2
2,2,344,Badminton Court
3,3,385,Table Tennis
4,4,421,Massage Room 1
5,5,27,Massage Room 2
6,6,195,Squash Court
7,7,421,Snooker Table
8,8,783,Pool Table


Q13: Find the facilities usage by month, but not guests

In [142]:
cur13 = conn.cursor()
    
query13 = """
SELECT 
    b.months, 
    COUNT(b.memid) AS usage
FROM 
    (SELECT MONTH(starttime) AS months, 
    memid
FROM Bookings
WHERE memid !=0) AS b
GROUP BY b.months;
    """
cur13.execute(query13)

OperationalError: no such function: MONTH

In [143]:
rows13 = cur11.fetchall()
rows13

[]

In [144]:
f13 = pd.DataFrame(rows13, columns=['months',
                                   'usage'])
f13

,months,usage
